* import scraping lib :

In [1]:
import json , csv
import scrapy as sc
import pandas as pd
from scrapy.crawler import CrawlerProcess as cp

* Setup Pipline :
*************
 * This class creates a simple pipeline that writes all found items to a JSON file, where each line contains one JSON element.



In [2]:
class JsonWriterPipeline(object) :
    def open_spider(self , spider) :
        self.file = open( "quoteresult.csv", 'w')
    def close_spider(self , spider) :
        self.file.close()
    def process_item(self , item ,spider) :
        line = json.dumps(dict(item)) + '\n'
        self.file.write(line)
        return item

* Define the spider
*****************
* The QuotesSpider class defines from which URLs to start crawling and which values to retrieve. I set the logging level of the crawler to warning, otherwise the notebook is overloaded with DEBUG messages about the retrieved data.



In [3]:
import logging as log

class QuotesSpider(sc.Spider) :
    name = 'quotes'
    starts_urls = ["http://quotes.toscrape.com/page/1/" ,
                   "http://quotes.toscrape.com/page/2/" ]
    custom_setting = {
        
        "LOG_Level" : log.WARNING , 
        "ITEM_PIPLINE" : {'__main__.JsonWriterPipeline' : 1} , 
        "FEED_FORMAT"  : 'csv' ,
        "FEED_URL"     : 'quoteresult.csv'
    }
def parser (self , res) :
    for quote in res.css('div.quote') :
        yield {
            'text'   : quote.css('span.text::text').extract_frist() ,
            'author' : quote.css('span small::text').extract_frist() ,
            'tags'   : quote.css('div.tags a.tag::text').extract(),
        }

* Start The Crawler

In [4]:
process = cp({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2021-10-04 20:54:11 [scrapy.utils.log] INFO: Scrapy 2.5.0 started (bot: scrapybot)
2021-10-04 20:54:11 [scrapy.utils.log] INFO: Versions: lxml 4.6.3.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 21.7.0, Python 3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 20.0.1 (OpenSSL 1.1.1k  25 Mar 2021), cryptography 3.4.7, Platform Windows-10-10.0.18362-SP0
2021-10-04 20:54:11 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.selectreactor.SelectReactor
2021-10-04 20:54:11 [scrapy.crawler] INFO: Overridden settings:
{'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2021-10-04 20:54:11 [scrapy.extensions.telnet] INFO: Telnet Password: 6696a64232af0370
2021-10-04 20:54:11 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.logstats.LogStats']
2021-10-04 20:54:11 [scrapy.middleware] INFO: Enabled downloader middlewares

In [5]:
!tail -n 2 quoteresult.jl


'tail' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!tail -n 2 quoteresult.json


'tail' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
df = pd.read_csv('quoteresult.csv' )
df

FileNotFoundError: [Errno 2] No such file or directory: 'quoteresult.csv'

In [8]:
dfjl = pd.read_json('quoteresult.jl', lines=True)
dfjl


ValueError: Expected object or value

In [ ]:
with open('quoteresult.json') as datafile:
    for line in datafile:
        data = json.loads(line)
retail = pd.DataFrame(data)